In [26]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType
import pandas as pd
import os
from pyspark.sql.functions import split
import sqlite3

In [27]:


schema = StructType([
    StructField("bookId", StringType()),
    StructField("userId", StringType()),
    StructField("rating", FloatType()),
    StructField("timestamp", TimestampType()),
    StructField("title", StringType())
])


spark = SparkSession.builder.appName("demo2").config(
    "spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3").getOrCreate()
# spark.sparkContext.setLogLevel("DEBUG")

streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "books") \
    .option("startingOffsets", "earliest") \
    .load()

con = sqlite3.connect('test.db')

cur = con.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS your_table (
    bookId TEXT,
    userId TEXT,
    rating REAL,
    timestamp INTEGER,
    title TEXT
)
"""


cur.execute(create_table_query)
con.commit()

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "books") \
    .option("startingOffsets", "earliest") \
    .load() \
    .select(from_json(col("value").cast("string"), schema=schema).alias("parsed_value"))


df_ = df.select("parsed_value.*")

writing_sink = df_.writeStream \
    .format("console") \
    .option("path", "./docs/jsonFiles") \
    .option("checkpointLocation", "./docs/jsonFiles/checkpoint") \
    .start()

writing_sink.stop()
con.commit()
cur.close()
con.close()

In [28]:
writing_df = df.writeStream.format("jdbc").option("url", "jdbc:sqlite:test.db").option("driver", "org.sqlite.JDBC").option("dbtable", "books")